# This notebook will generate the Plausibility and Faithfulness results for the models

## The Expainability metrices are based on the work by DeYoung et. al. (2020) [ERASER: A Benchmark to Evaluate Rationalized NLP Models]

In [18]:
# model choices include 'bert','bert_supervised','birnn','cnngru','birnn_att','birnn_scrat'
# attention lambda for bert_supervised is 0.001 and birnn_scrat is 100
# for testing_with_lime.py set number of samples is 100 (for faster inference set it lower but the results might not be consistent)



#python testing_with_rational.py <model> <att_lambda>
!python testing_with_rational.py bert_supervised 0.001
#python testing_with_lime.py <model> <number of samples> <att_lambda>
!python testing_with_lime.py bert_supervised 100 0.001

2024-04-30 02:46:24.585169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-30 02:46:24.585212: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/tchuente/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading english - 1grams ...
Since you dont want to use GPU, using the CPU instead.
/home/t

Loading BERT tokenizer...
total_data 1143
100%|█████████████████████████████████████| 1143/1143 [00:00<00:00, 2780.20it/s]
/home/tchuente/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Running eval on test data...
  0%|                                                    | 0/72 [00:00<?, ?it/s]Traceback (most recent call last):
  File "testing_with_rational.py", line 390, in <module>
    final_list_dict=get_final_dict_with_rational(params, params['data_file'],topk=5)
  File "testing_with_rational.py", line 287, in get_final_dict_with_rational
    list_dict_org,test_data=standaloneEval_with_rational(params, extra_data_path=test_data,topk=topk)
  File "testing_with_rational.py", line 200, in standaloneEval_with_rational
    labels=None,device=device)
  File "/h

total_data 100

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 148.66it/s]

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 3727.38it/s]
Running eval on test data...
  0%|                                                  | 0/1925 [00:09<?, ?it/s]
Traceback (most recent call last):
  File "testing_with_lime.py", line 404, in <module>
    final_dict=get_final_dict_with_lime(params,model_to_use,test_data,topk=5)
  File "testing_with_lime.py", line 321, in get_final_dict_with_lime
    list_dict_org,test_data=standaloneEval_with_lime(params,model_name,test_data=test_data, topk=topk)
  File "testing_with_lime.py", line 272, in standaloneEval_with_lime
    exp = explainer.explain_instance(sentence, modelClass.return_probab, num_features=6, top_labels=3,num_samples=params['num_samples'])
  File "/home/tchuente/.local/lib/python3.6/site-packages/lime/lime_text.py", line 415, in explain_instance
    distance_metric=distance_metric)
  File "/home/tchuente

## <font color='red'>NOTE: Please generate the model explainability output before running this notebook</font>

In [2]:
import json
from tqdm.notebook import tqdm
import more_itertools as mit
import os

In [3]:
# get_annotated_data method is used to load the dataset
from Preprocess import *
from Preprocess.dataCollect import *

/home/tchuente/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading english - 1grams ...


In [4]:
dict_data_folder={
      '2':{'data_file':'Data/dataset.json','class_label':'Data/classes_two.npy'},
      '3':{'data_file':'Data/dataset.json','class_label':'Data/classes.npy'}
}

# We need to load the dataset with the labels as 'hatespeech', 'offensive', and 'normal' (3-class). 

params = {}
params['num_classes']=3
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)

In [5]:
# The important key here is the 'bert_token'. Set it to True for Bert based models and False for Others.

params_data={
    'include_special':False,  #True is want to include <url> in place of urls if False will be removed
    'bert_tokens':True, #True /False
    'type_attention':'softmax', #softmax
    'set_decay':0.1,
    'majority':2,
    'max_length':128,
    'variance':10,
    'window':4,
    'alpha':0.5,
    'p_value':0.8,
    'method':'additive',
    'decay':False,
    'normalized':False,
    'not_recollect':True,
}


if(params_data['bert_tokens']):
    print('Loading BERT tokenizer...')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)
else:
    print('Loading Normal tokenizer...')
    tokenizer=None

Loading BERT tokenizer...


In [6]:
# Load the whole dataset and get the tokenwise rationales
def get_training_data(data):
    post_ids_list=[]
    text_list=[]
    attention_list=[]
    label_list=[]
    
    final_binny_output = []
    print('total_data',len(data))
    for index,row in tqdm(data.iterrows(),total=len(data)):
        annotation=row['final_label']
        
        text=row['text']
        post_id=row['post_id']
        annotation_list=[row['label1'],row['label2'],row['label3']]
        tokens_all = list(row['text'])
#         attention_masks =  [list(row['explain1']),list(row['explain2']),list(row['explain1'])]
        
        if(annotation!= 'undecided'):
            tokens_all,attention_masks=returnMask(row, params_data, tokenizer)
            final_binny_output.append([post_id, annotation, tokens_all, attention_masks, annotation_list])

    return final_binny_output



In [7]:
training_data=get_training_data(data_all_labelled)

  0%|          | 11/20149 [00:00<03:11, 104.97it/s]

total_data 20149


100%|██████████| 20149/20149 [01:49<00:00, 183.72it/s]


In [8]:
training_data[19]

['18696652_gab',
 'offensive',
 [101,
  5310,
  25805,
  5582,
  4319,
  2224,
  2025,
  11382,
  3489,
  2012,
  2035,
  4283,
  2005,
  2008,
  19380,
  102],
 [[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]],
 ['hatespeech', 'offensive', 'offensive']]

In [9]:
# https://stackoverflow.com/questions/2154249/identify-groups-of-continuous-numbers-in-a-list
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]
            
# Convert dataset into ERASER format: https://github.com/jayded/eraserbenchmark/blob/master/rationale_benchmark/utils.py
def get_evidence(post_id, anno_text, explanations):
    output = []

    indexes = sorted([i for i, each in enumerate(explanations) if each==1])
    span_list = list(find_ranges(indexes))

    for each in span_list:
        if type(each)== int:
            start = each
            end = each+1
        elif len(each) == 2:
            start = each[0]
            end = each[1]+1
        else:
            print('error')

        output.append({"docid":post_id, 
              "end_sentence": -1, 
              "end_token": end, 
              "start_sentence": -1, 
              "start_token": start, 
              "text": ' '.join([str(x) for x in anno_text[start:end]])})
    return output

# To use the metrices defined in ERASER, we will have to convert the dataset
def convert_to_eraser_format(dataset, method, save_split, save_path, id_division):  
    final_output = []
    
    if save_split:
        train_fp = open(save_path+'train.jsonl', 'w')
        val_fp = open(save_path+'val.jsonl', 'w')
        test_fp = open(save_path+'test.jsonl', 'w')
            
    for tcount, eachrow in enumerate(dataset):
        
        temp = {}
        post_id = eachrow[0]
        post_class = eachrow[1]
        anno_text_list = eachrow[2]
        majority_label = eachrow[1]
        
        if majority_label=='normal':
            continue
        
        all_labels = eachrow[4]
        explanations = []
        for each_explain in eachrow[3]:
            explanations.append(list(each_explain))
        
        # For this work, we have considered the union of explanations. Other options could be explored as well.
        if method == 'union':
            final_explanation = [any(each) for each in zip(*explanations)]
            final_explanation = [int(each) for each in final_explanation]
        
            
        temp['annotation_id'] = post_id
        temp['classification'] = post_class
        temp['evidences'] = [get_evidence(post_id, list(anno_text_list), final_explanation)]
        temp['query'] = "What is the class?"
        temp['query_type'] = None
        final_output.append(temp)
        
        if save_split:
            if not os.path.exists(save_path+'docs'):
                os.makedirs(save_path+'docs')
            
            with open(save_path+'docs/'+post_id, 'w') as fp:
                fp.write(' '.join([str(x) for x in list(anno_text_list)]))
            
            if post_id in id_division['train']:
                train_fp.write(json.dumps(temp)+'\n')
            
            elif post_id in id_division['val']:
                val_fp.write(json.dumps(temp)+'\n')
            
            elif post_id in id_division['test']:
                test_fp.write(json.dumps(temp)+'\n')
            else:
                print(post_id)
    
    if save_split:
        train_fp.close()
        val_fp.close()
        test_fp.close()
        
    return final_output

In [11]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
with open('./Data/post_id_divisions.json') as fp:
    id_division = json.load(fp)

In [12]:
method = 'union'
save_split = True
save_path = './Data/Evaluation/Model_Eval/'  #The dataset in Eraser Format will be stored here.
convert_to_eraser_format(training_data, method, save_split, save_path, id_division)

[{'annotation_id': '1179088797964763136_twitter',
  'classification': 'hatespeech',
  'evidences': [[{'docid': '1179088797964763136_twitter',
     'end_sentence': -1,
     'end_token': 21,
     'start_sentence': -1,
     'start_token': 19,
     'text': '19992 24008'}]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '1179085312976445440_twitter',
  'classification': 'hatespeech',
  'evidences': [[{'docid': '1179085312976445440_twitter',
     'end_sentence': -1,
     'end_token': 3,
     'start_sentence': -1,
     'start_token': 2,
     'text': '7743'},
    {'docid': '1179085312976445440_twitter',
     'end_sentence': -1,
     'end_token': 25,
     'start_sentence': -1,
     'start_token': 20,
     'text': '1045 5223 2317 7743 2229'}]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '13851720_gab',
  'classification': 'hatespeech',
  'evidences': [[{'docid': '13851720_gab',
     'end_sentence': -1,
     'end_token': 17,
     'start

### Generate the output

In [13]:
%cd eraserbenchmark-master

[Errno 2] No such file or directory: 'eraserbenchmark-master'
/home/tchuente/Téléchargements/HateXplain-master-2


In [14]:
!ls

1.png
2.png
bestModel_birnnscrat_100_explanation_top5_max.json
bestModel_birnnscrat_100_explanation_top5_min.json
bestModel_birnnscrat_100_explanation_top5_normal.json
best_model_json
best_runs.sh
Bias_Calculation_NB.ipynb
conda
convert_to_word2vec.py
Data
eraserbenchmark
Example_HateExplain.ipynb
Example_HateExplain_v1.ipynb
Explainability_Calculation_NB.ipynb
explanations_dicts
Figures
LICENSE
manual_training_inference.py
model_explain_output.json
Models
Models_and_results
Parameters_description.md
parameters_selection.py
Pipfile
Preprocess
__pycache__
Question_master.txt
README.md
requirements.txt
Saved
TensorDataset
testing_for_bias.py
testing_with_lime.py
testing_with_rational.py
test_parallel.sh
Untitled.ipynb


In [16]:
#--data_dir : Location of the folder which contains the dataset in eraser format
#--results : The location of the model output file in eraser format
#--score_file : The file name and location to write the output

!PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --strict --data_dir ../Data/Evaluation/Model_Eval --results ../explanations_dicts/bestModel_bert_base_uncased_Attn_train_TRUE_0.001_explanation_top5.json --score_file ../model_explain_output.json

python: can't open file 'rationale_benchmark/metrics.py': [Errno 2] No such file or directory


In [17]:
# print the required results
with open('../model_explain_output.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])

FileNotFoundError: [Errno 2] No such file or directory: '../model_explain_output.json'